In [14]:
# Set env var OPENAI_API_KEY or load from a .env file
import dotenv

dotenv.load_dotenv()

True

In [15]:
os.environ["LANGSMITH_TRACING"] = "false"

In [16]:
import os

my_test = os.getenv("LANGSMITH_TRACING")
print(my_test)

false


In [22]:
from langchain_community.document_loaders import WebBaseLoader

doc_url = "https://langchain-ai.github.io/langgraph/troubleshooting/errors/" #"https://python.langchain.com/docs/contributing/" #"https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(doc_url)
docs = loader.load()

print(docs)

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/troubleshooting/errors/', 'title': 'Troubleshooting', 'description': 'Build language agents as graphs', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\nTroubleshooting\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\nJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            \n              Troubleshooting\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n    \n  \n  Home\n\n        \n\n\n\n          \n  \n    \n  \n  API reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n    \n  \n    Home\n  \n\n    \n  \n\n\n\n\n\n\n\n\n            \n  \n    Home\n  \n\n    

In [23]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2", temperature=0)

## Summarize in a single LLM call

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import  LLMChain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\n\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

In [29]:
for token in chain.stream({"context": docs}):
    print(token, end="|")

|

## Map-Reduce

In [30]:
from langchain_core.prompts import ChatPromptTemplate

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

In [31]:
from langchain import hub

map_prompt = hub.pull("rlm/map-prompt")

In [32]:
# Also available via the hub: `hub.pull("rlm/reduce-prompt")`
reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

In [35]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0)

split_docs = text_splitter.split_documents(docs)
print(f"generated {len(split_docs)} documents")

generated 3 documents


In [38]:
# Invoke chain
result = chain.invoke({"context": [split_docs[0]]})
print(result)

In [37]:
for token in chain.stream({"context": split_docs}):
    print(token, end="|")

|